In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import time # For simulation delay
import os   # To read environment variables
import requests # To make API calls (install if needed: pip install requests)
import json # To handle JSON responses

# --- Define Oasis Options ---
# Dictionary mapping user-friendly names to actual API URLs
oasis_options = {
    "SE Oasis":"https://nomad-hzb-se.de/nomad-oasis/api/v1",
    "CE Oasis":"https://nomad-hzb-ce.de/nomad-oasis/api/v1",
    "Sol-AI Oasis":"https://nomad-sol-ai.de/nomad-oasis/api/v1",
}

# --- Application State ---
# Store the currently active token after successful authentication
current_token = None
# Store the authenticated user info (optional, but useful)
current_user_info = None

# --- General Settings Widgets ---

# Dropdown for selecting NOMAD Oasis by name
# Ensure there's at least one option, otherwise widget creation might fail
if not oasis_options:
    oasis_options["Default (Please Edit Code)"] = "http://example.com/api/v1"

oasis_dropdown = widgets.Dropdown(
    options=oasis_options.keys(),
    value=list(oasis_options.keys())[0], # Default to the first option
    description='Select Oasis:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(min_width='250px', flex='1 1 auto')
)

# Label to display the URL of the selected Oasis (read-only)
selected_url_display = widgets.Label(
    # Handle case where oasis_options might be initially empty
    value=f"Base URL: {oasis_options.get(oasis_dropdown.value, 'N/A')}",
    layout=widgets.Layout(margin='5px 0 5px 5px', width='auto') # Added bottom margin
)

# --- Authentication Widgets ---

# Radio buttons to select authentication method
auth_method_selector = widgets.RadioButtons(
    options=['Username/Password', 'Token (from ENV)'], # Updated label for clarity
    description='Auth Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px 0 0 0')
)

# --- Username/Password Widgets ---
username_input = widgets.Text(
    placeholder='Enter Username (e.g., email)',
    description='Username:',
    style={'description_width': 'initial'}
)
password_input = widgets.Password(
    placeholder='Enter Password',
    description='Password:',
    style={'description_width': 'initial'}
)
# Box to hold U/P inputs, initially hidden/shown based on environment
local_auth_box = widgets.VBox([username_input, password_input], layout=widgets.Layout(margin='5px 0 0 0'))

# --- Token Widget (Hidden when Token method selected, but defined for layout) ---
token_input = widgets.Password(
    placeholder='Token will be read from ENV', # Updated placeholder
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='95%'),
    disabled=True # Disable input as it's not used directly
)
# Box to hold token input, initially hidden/shown based on environment
# This box will now be hidden when 'Token (from ENV)' is selected
token_auth_box = widgets.VBox([token_input], layout=widgets.Layout(margin='5px 0 0 0', display='none')) # Start hidden

# --- Authentication Action Widgets ---
auth_button = widgets.Button(
    description='Authenticate',
    button_style='info',
    tooltip='Authenticate using the selected method',
    icon='unlock',
    layout=widgets.Layout(width='auto', margin='10px 0 0 0') # Add top margin
)

auth_status_label = widgets.Label(
    value='Status: Not Authenticated',
    layout=widgets.Layout(margin='5px 0 0 0')
)

# Box to hold the button and status
auth_action_box = widgets.VBox([auth_button, auth_status_label])

# --- Observers and Initial Setup ---

# Observer for Oasis change
def on_oasis_change(change):
    global current_token, current_user_info # Allow modification of global state
    if change['type'] == 'change' and change['name'] == 'value':
        selected_name = change['new']
        selected_url = oasis_options.get(selected_name, "Invalid selection - URL not found")
        selected_url_display.value = f"Base URL: {selected_url}"
        # Reset authentication status and token when Oasis changes
        auth_status_label.value = 'Status: Not Authenticated (Oasis changed)'
        auth_status_label.style.text_color = None
        current_token = None
        current_user_info = None

oasis_dropdown.observe(on_oasis_change, names='value')

# Observer for Auth Method change
def on_auth_method_change(change):
    global current_token, current_user_info # Allow modification of global state
    if change['type'] == 'change' and change['name'] == 'value':
        selected_method = change['new']
        # Show/hide relevant input boxes based on selection
        if selected_method == 'Username/Password':
            local_auth_box.layout.display = 'flex' # Show U/P box
            token_auth_box.layout.display = 'none'  # Hide Token box (remains hidden)
        else: # Token (from ENV) method selected
            local_auth_box.layout.display = 'none'  # Hide U/P box
            token_auth_box.layout.display = 'none' # Hide Token box as input is not needed

        # Reset authentication status and token when method changes
        auth_status_label.value = 'Status: Not Authenticated (Method changed)'
        auth_status_label.style.text_color = None
        current_token = None
        current_user_info = None

auth_method_selector.observe(on_auth_method_change, names='value')

# --- Initial Environment Detection ---
# Check if running in a known JupyterHub environment to set default auth method
is_hub_environment = bool(os.environ.get('JUPYTERHUB_USER'))

if is_hub_environment:
    # Default to Token method if in Hub, assuming token ENV var might be set
    auth_method_selector.value = 'Token (from ENV)'
    local_auth_box.layout.display = 'none'
    token_auth_box.layout.display = 'none' # Ensure token box is hidden
else:
    # Default to Username/Password otherwise
    auth_method_selector.value = 'Username/Password'
    local_auth_box.layout.display = 'flex'
    token_auth_box.layout.display = 'none' # Ensure token box is hidden


# --- Arrange General Settings Widgets ---
# Combine all authentication widgets vertically
auth_box = widgets.VBox([
    auth_method_selector,
    local_auth_box,  # Will be shown/hidden by observer
    token_auth_box,  # Will be hidden by observer when Token method selected
    auth_action_box
])

# Main settings box layout
general_settings_box = widgets.VBox([
    widgets.HTML("<h2>General Settings</h2>"),
    oasis_dropdown,
    selected_url_display,
    auth_box # Add the combined authentication section
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='0 0 20px 0'))

# --- Authentication Button Functionality ---
def on_auth_button_clicked(b):
    global current_token, current_user_info # Allow modification of global state
    # Reset state before attempting authentication
    current_token = None
    current_user_info = None
    auth_status_label.value = 'Status: Authenticating...'
    auth_status_label.style.text_color = 'orange'

    selected_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_name)
    auth_method = auth_method_selector.value

    if not base_url:
        auth_status_label.value = 'Status: Error - Invalid Oasis selected.'
        auth_status_label.style.text_color = 'red'
        return

    try:
        if auth_method == 'Username/Password':
            # Get username and password from input widgets
            username = username_input.value
            password = password_input.value
            if not username or not password:
                raise ValueError("Username and Password are required.")

            # Create the dictionary structure as requested by the user (optional here)
            auth_dict = dict(username=username, password=password)

            token_url = f"{base_url}/auth/token"

            # *** CORRECTED LINE BELOW ***
            # Send username/password as query parameters using the 'params' argument
            # Remove the 'auth' argument which sends Basic Auth header
            response = requests.get(token_url, params=auth_dict, timeout=10) # Use params= instead of auth=
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

            token_data = response.json()
            if 'access_token' not in token_data:
                 raise ValueError("Access token not found in response.")

            current_token = token_data['access_token']
            # Clear password field after successful U/P auth for security
            password_input.value = ''

        elif auth_method == 'Token (from ENV)':
            # Read token directly from the specified environment variable
            token_from_env = os.environ.get('NOMAD_CLIENT_ACCESS_TOKEN')
            if not token_from_env:
                # Raise error if the environment variable is not set or empty
                raise ValueError("Token not found in environment variable 'NOMAD_CLIENT_ACCESS_TOKEN'.")
            current_token = token_from_env # Use the token from ENV

        # --- Verify Token (applies to both methods after obtaining a token) ---
        if current_token:
            verify_url = f"{base_url}/users/me"
            headers = {'Authorization': f'Bearer {current_token}'}
            verify_response = requests.get(verify_url, headers=headers, timeout=10)
            verify_response.raise_for_status() # Check if verification call was successful

            current_user_info = verify_response.json() # Store user info
            user_display = current_user_info.get('name', current_user_info.get('username', 'Unknown User'))
            auth_status_label.value = f'Status: Authenticated as {user_display} on {selected_name}.'
            auth_status_label.style.text_color = 'green'

        else:
             # This case should only be reached if something unexpected happened before token assignment
             raise ValueError("Authentication failed, no token available for verification.")

    except ValueError as ve: # Handle validation errors (missing input, missing ENV var)
         auth_status_label.value = f'Status: Error - {ve}'
         auth_status_label.style.text_color = 'red'
         current_token = None # Ensure token is cleared on error
         current_user_info = None
    except requests.exceptions.RequestException as e: # Handle network/HTTP errors
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                 # Try to get specific error detail from JSON response
                 error_detail = e.response.json().get('detail', e.response.text)
                 # Handle cases where detail might be a list (like the 422 error)
                 if isinstance(error_detail, list):
                     error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                 else:
                     error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                 # Fallback if response is not JSON
                 error_message = f"API Error ({e.response.status_code}): {e.response.text}"

        auth_status_label.value = f'Status: {error_message}'
        auth_status_label.style.text_color = 'red'
        current_token = None # Ensure token is cleared on error
        current_user_info = None
    except Exception as e: # Catch any other unexpected errors
        auth_status_label.value = f'Status: Unexpected Error - {e}'
        auth_status_label.style.text_color = 'red'
        current_token = None
        current_user_info = None


# Link the button's on_click event to the updated function
auth_button.on_click(on_auth_button_clicked)

# --- Note ---
# Make sure you have the 'requests' library installed (`pip install requests`)
# When 'Token (from ENV)' method is selected, the token is read from the
# 'NOMAD_CLIENT_ACCESS_TOKEN' environment variable upon clicking 'Authenticate'.
# The token endpoint for U/P ('/auth/token') uses a GET request and expects
# username/password as query parameters (sent via 'params' dict).
# The token verification endpoint is assumed to be '/users/me'. Adjust if necessary.


In [2]:
# --- State Variables for this Tab ---
# Store group data fetched from API for the visualize tab
group_data_store = {} # {group_id: group_info, ...}
group_name_to_id_map = {} # {group_name: group_id, ...}

# --- Helper Function for API Calls ---
# Note: If make_api_request is defined globally in your main script, you can remove this.
# This helper function needs access to the 'current_token' variable from the main scope.
def make_api_request(method, url, headers=None, params=None, json_data=None, timeout=10):
    """Makes an API request and handles common errors."""
    # Use the global current_token from the main dashboard scope
    if not current_token:
         raise ValueError("Authentication token is missing.")

    # Prepare headers, ensuring the token is included
    request_headers = headers.copy() if headers else {}
    request_headers['Authorization'] = f'Bearer {current_token}'

    try:
        response = requests.request(method, url, headers=request_headers, params=params, json=json_data, timeout=timeout)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        if response.text:
             return response.json()
        return None # Return None for empty successful responses (e.g., 204 No Content)
    except requests.exceptions.RequestException as e:
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get('detail', e.response.text)
                if isinstance(error_detail, list):
                    error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                else:
                    error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                error_message = f"API Error ({e.response.status_code}): {e.response.text}"
        raise ConnectionError(error_message) from e
    except Exception as e:
        raise Exception(f"Unexpected Error during API request: {e}") from e


# --- Visualize Members Tab Widgets ---
load_groups_button = widgets.Button(
    description="Load Groups",
    button_style='primary',
    tooltip='Fetch available groups from the selected Oasis',
    icon='refresh'
)

groups_dropdown = widgets.Dropdown(
    description="Select Group:",
    options=[], # Initially empty
    disabled=True, # Disabled until groups are loaded
    style={'description_width': 'initial'}
)

# ADDED: Button to trigger loading members for the selected group
load_members_button = widgets.Button(
    description="Load Members",
    button_style='info', # Use 'info' or another style
    tooltip='Load members for the selected group',
    icon='users', # FontAwesome icon for users
    disabled=True # Start disabled, enable when group is selected
)

# Output widget to display messages, progress, and member list
members_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# --- Handler Functions for Visualize Tab ---

def on_load_groups_clicked(b):
    """Fetches groups from the API and populates the dropdown."""
    global group_data_store, group_name_to_id_map # Allow modification of tab state

    # Access main dashboard state for token and URL
    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist in the parent scope
    if not current_token:
         with members_output:
            clear_output(wait=True)
            print("Authentication required. Please authenticate first in General Settings.")
         return

    selected_oasis_name = oasis_dropdown.value # Get value from main dropdown
    base_url = oasis_options.get(selected_oasis_name) # Get URL from main options dict
    if not base_url:
         with members_output:
            clear_output(wait=True)
            print("Error: Invalid Oasis selected in General Settings.")
         return

    with members_output:
        clear_output(wait=True)
        print(f"Loading groups from {selected_oasis_name}...")
        # Disable controls while loading
        groups_dropdown.disabled = True
        groups_dropdown.options = []
        groups_dropdown.value = None # Reset selection
        load_members_button.disabled = True # Ensure members button is disabled
        group_data_store = {}
        group_name_to_id_map = {}

        try:
            groups_url = f"{base_url}/groups"
            groups_response = make_api_request('get', groups_url, params={'page_size': 1000}) # Use helper

            groups = groups_response.get('data', [])
            if not groups:
                 print("No groups found for the authenticated user.")
                 return # Keep controls disabled

            # Store data and populate dropdown
            group_data_store = {g['group_id']: g for g in groups}
            group_name_to_id_map = {g['group_name']: g['group_id'] for g in groups}
            dropdown_options = sorted(group_name_to_id_map.keys())

            groups_dropdown.options = dropdown_options
            groups_dropdown.disabled = False # Enable dropdown
            # Keep load_members_button disabled until a group is selected
            print(f"Loaded {len(groups)} groups. Please select one from the dropdown.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"Error loading groups: {e}")
            # Ensure controls are reset/disabled on error
            groups_dropdown.options = []
            groups_dropdown.disabled = True
            load_members_button.disabled = True


def fetch_user_details(user_id, base_url):
    """Helper to get user details (Name, Last Name). Needs access to current_token via make_api_request."""
    # Assumes 'make_api_request' is defined and uses 'current_token' from parent scope
    try:
        user_url = f"{base_url}/users/{user_id}"
        user_data = make_api_request('get', user_url) # Use helper

        first_name = user_data.get('first_name', '')
        last_name = user_data.get('last_name', '')
        name = f"{first_name} {last_name}".strip()

        if not name: # Fallback if name fields are empty
            name = user_data.get('username', user_data.get('email', f"ID: {user_id} (No name/email)"))
        return name
    except (ConnectionError, ValueError, Exception) as e:
        # Log error within the output area for context
        print(f"  Error fetching details for user {user_id}: {e}")
        return f"ID: {user_id} (Error)"


def on_group_selected(change):
    """Handles group selection: enables/disables the Load Members button."""
    # Check if the change is relevant and a valid group name is selected
    if change['type'] == 'change' and change['name'] == 'value':
        selected_group_name = change['new']

        # Enable button if a group is selected, disable otherwise
        if selected_group_name:
            load_members_button.disabled = False
            # Clear previous member list when selection changes, prompt user to load
            with members_output:
                clear_output(wait=True)
                print(f"Group '{selected_group_name}' selected. Click 'Load Members'.")
        else:
            load_members_button.disabled = True
            # Clear output if selection is reset
            with members_output:
                clear_output(wait=True)


# NEW Handler for the Load Members button
def on_load_members_clicked(b):
    """Fetches and displays members for the currently selected group."""
    # Access main dashboard state
    if not current_token:
         with members_output:
            clear_output(wait=True)
            print("Authentication required.")
         return

    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with members_output:
            clear_output(wait=True)
            print("Error: Invalid Oasis selected.")
         return

    # Get the currently selected group from the dropdown
    selected_group_name = groups_dropdown.value
    if not selected_group_name:
        with members_output:
            clear_output(wait=True)
            print("Error: No group selected.")
        return

    # Get the group ID using the map populated earlier
    group_id = group_name_to_id_map.get(selected_group_name)
    if not group_id:
         with members_output:
            clear_output(wait=True)
            print(f"Error: Could not find ID for group '{selected_group_name}'")
         return

    # Fetch and display members
    with members_output:
        clear_output(wait=True)
        print(f"Fetching members for group '{selected_group_name}' (ID: {group_id})...")
        load_members_button.disabled = True # Disable button while loading
        try:
            group_details_url = f"{base_url}/groups/{group_id}"
            group_details = make_api_request('get', group_details_url) # Use helper

            # *** Assumption: Adjust 'members' key if the API uses a different field name ***
            member_ids = group_details.get('members', [])

            if not member_ids:
                print("No members found in this group.")
                load_members_button.disabled = False # Re-enable button
                return

            print(f"Found {len(member_ids)} members. Fetching details...")
            member_names = []
            # Fetch details one by one
            for i, user_id in enumerate(member_ids):
                print(f"  Fetching user {i+1}/{len(member_ids)} (ID: {user_id})...") # Progress
                member_name = fetch_user_details(user_id, base_url)
                member_names.append(member_name)
                time.sleep(0.05) # Optional small delay

            print("\n--- Group Members ---")
            for name in sorted(member_names): # Sort alphabetically
                print(f"- {name}")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError fetching members for group '{selected_group_name}': {e}")
        finally:
             # Re-enable button after loading attempt (success or failure)
             # Check if a group is still selected before enabling
             if groups_dropdown.value:
                  load_members_button.disabled = False


# --- Define Visualize Tab Layout ---
# This VBox contains all the widgets and structure for this specific tab
visualize_tab = widgets.VBox([
    load_groups_button,
    groups_dropdown,
    load_members_button, # Added the new button here
    widgets.HTML("<hr style='margin: 10px 0;'><b>Members:</b>"), # Separator and label
    members_output
])

# --- Connect Handlers ---
# These lines connect the button click and dropdown change events to their functions
load_groups_button.on_click(on_load_groups_clicked)
groups_dropdown.observe(on_group_selected, names='value')
load_members_button.on_click(on_load_members_clicked) # Connect the new button


# --- Integration Note ---
# The 'visualize_tab' VBox defined above should be assigned as a child
# to the main Tab widget in your dashboard script. For example:
# main_tab_widget.children = [visualize_tab, add_member_tab, ...]

In [3]:
# --- State Variables for this Tab (if needed) ---
# Using a separate map in case loading is independent from visualize tab
add_tab_group_name_to_id_map = {}

# --- Helper Function for API Calls ---
# Note: If make_api_request is defined globally in your main script, you can remove this.
def make_api_request(method, url, headers=None, params=None, json_data=None, timeout=10):
    """Makes an API request and handles common errors."""
    # Use the global current_token from the main dashboard scope
    if not current_token:
         raise ValueError("Authentication token is missing.")

    # Prepare headers, ensuring the token is included
    request_headers = headers.copy() if headers else {}
    request_headers['Authorization'] = f'Bearer {current_token}'

    try:
        response = requests.request(method, url, headers=request_headers, params=params, json=json_data, timeout=timeout)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        # Handle potential empty responses on success (e.g., 204 No Content)
        # POST /edit returns 200 OK with body according to docs, so check text
        if response.text:
             return response.json()
        return None # Should not happen for POST /edit if successful based on docs
    except requests.exceptions.RequestException as e:
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get('detail', e.response.text)
                if isinstance(error_detail, list):
                    error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                else:
                    error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                error_message = f"API Error ({e.response.status_code}): {e.response.text}"
        raise ConnectionError(error_message) from e
    except Exception as e:
        raise Exception(f"Unexpected Error during API request: {e}") from e


# --- Add Member Tab Widgets ---
add_load_groups_button = widgets.Button(
    description="Load Groups",
    button_style='primary',
    tooltip='Fetch groups to populate the dropdown',
    icon='refresh'
)

add_groups_dropdown = widgets.Dropdown(
    description="Target Group:",
    options=[],
    disabled=True,
    style={'description_width': 'initial'}
)

members_input_area = widgets.Textarea(
    placeholder="Enter member emails, one per line...",
    layout=widgets.Layout(height='100px', width='auto')
)

add_members_button = widgets.Button(
    description="Add Members",
    button_style='success',
    tooltip='Add the entered emails as members to the selected group',
    icon='user-plus', # FontAwesome icon
    disabled=True # Disabled until group selected and emails entered
)

add_members_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# --- Handler Functions for Add Member Tab ---

def on_add_load_groups_clicked(b):
    """Fetches groups and populates the dropdown for the Add Member tab."""
    global add_tab_group_name_to_id_map

    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist in the parent scope
    if not current_token:
         with add_members_output:
            clear_output(wait=True)
            print("Authentication required.")
         return

    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with add_members_output:
            clear_output(wait=True)
            print("Error: Invalid Oasis selected.")
         return

    with add_members_output:
        clear_output(wait=True)
        print(f"Loading groups from {selected_oasis_name}...")
        add_groups_dropdown.disabled = True
        add_groups_dropdown.options = []
        add_groups_dropdown.value = None
        add_members_button.disabled = True # Disable add button too
        add_tab_group_name_to_id_map = {}

        try:
            groups_url = f"{base_url}/groups"
            groups_response = make_api_request('get', groups_url, params={'page_size': 1000}) # Use helper

            groups = groups_response.get('data', [])
            if not groups:
                 print("No groups found.")
                 return

            add_tab_group_name_to_id_map = {g['group_name']: g['group_id'] for g in groups}
            dropdown_options = sorted(add_tab_group_name_to_id_map.keys())

            add_groups_dropdown.options = dropdown_options
            add_groups_dropdown.disabled = False
            print(f"Loaded {len(groups)} groups. Select a target group.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"Error loading groups: {e}")
            add_groups_dropdown.options = []
            add_groups_dropdown.disabled = True


def check_add_button_state(*args):
    """Enable Add Members button only if a group is selected and emails are entered."""
    group_selected = bool(add_groups_dropdown.value)
    emails_entered = bool(members_input_area.value.strip())
    add_members_button.disabled = not (group_selected and emails_entered)

# Observe changes in dropdown and textarea to control button state
add_groups_dropdown.observe(check_add_button_state, names='value')
members_input_area.observe(check_add_button_state, names='value')


def on_add_members_clicked(b):
    """Handles adding members to the selected group."""
    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist in the parent scope
    if not current_token:
         with add_members_output:
            clear_output(wait=True); print("Authentication required.")
         return

    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with add_members_output:
            clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    # Get selected group and emails
    selected_group_name = add_groups_dropdown.value
    group_id = add_tab_group_name_to_id_map.get(selected_group_name)
    emails_text = members_input_area.value.strip()

    if not group_id:
        with add_members_output: clear_output(wait=True); print("Error: Please select a valid group.")
        return
    if not emails_text:
        with add_members_output: clear_output(wait=True); print("Error: Please enter emails to add.")
        return

    emails_to_add = [email.strip() for email in emails_text.splitlines() if email.strip()]
    if not emails_to_add:
        with add_members_output: clear_output(wait=True); print("Error: No valid emails entered.")
        return

    with add_members_output:
        clear_output(wait=True)
        print(f"Adding members to group '{selected_group_name}'...")
        add_members_button.disabled = True # Disable while processing

        found_user_ids = []
        not_found_emails = []

        # 1. Look up User IDs by email (one by one)
        print("\n1. Looking up users by email...")
        users_url = f"{base_url}/users"
        for i, email in enumerate(emails_to_add):
            print(f"  Checking email {i+1}/{len(emails_to_add)}: {email}...")
            try:
                # Assuming API supports filtering users by email via query param
                user_response = make_api_request('get', users_url, params={'email': email}) # Use helper
                user_data = user_response.get('data', []) if user_response else []

                if user_data and len(user_data) > 0:
                    user_id = user_data[0].get('user_id')
                    if user_id:
                        found_user_ids.append(user_id)
                        print(f"    Found User ID: {user_id}")
                    else:
                        not_found_emails.append(email)
                        print("    Error: Found user data but no user_id.")
                else:
                    not_found_emails.append(email)
                    print("    User not found.")
            except (ConnectionError, ValueError, Exception) as e:
                print(f"    Error looking up email {email}: {e}")
                not_found_emails.append(f"{email} (Lookup Error)")
            time.sleep(0.05) # Small delay

        # Report users not found
        if not_found_emails:
            print("\nWarning: Could not find or process the following emails:")
            for email in not_found_emails:
                print(f"- {email}")

        if not found_user_ids:
            print("\nError: No valid users found for the provided emails. Aborting.")
            add_members_button.disabled = False # Re-enable button
            return

        # 2. Fetch Existing Members
        print("\n2. Fetching existing members...")
        existing_member_ids = []
        try:
            group_details_url = f"{base_url}/groups/{group_id}"
            group_details = make_api_request('get', group_details_url) # Use helper
            existing_member_ids = group_details.get('members', [])
            print(f"  Found {len(existing_member_ids)} existing members.")
        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError fetching existing members: {e}. Aborting.")
            add_members_button.disabled = False # Re-enable button
            return

        # 3. Combine Member Lists (ensure uniqueness)
        print("\n3. Combining member lists...")
        existing_set = set(existing_member_ids)
        combined_ids_set = set(existing_member_ids) | set(found_user_ids)

        # Calculate how many are actually new additions
        newly_added_ids = set(found_user_ids) - existing_set
        newly_added_count = len(newly_added_ids)

        if newly_added_count == 0:
             print("\nAll specified users are already members of the group. No changes made.")
             add_members_button.disabled = False # Re-enable button
             return

        final_member_list = list(combined_ids_set)
        print(f"  Total members after addition: {len(final_member_list)} ({newly_added_count} new).")

        # 4. Update Group Members (POST /edit request)
        print("\n4. Updating group membership via POST /edit...")
        try:
            # *** CORRECTED Endpoint and Method based on API Docs ***
            edit_url = f"{base_url}/groups/{group_id}/edit"
            payload = {"members": final_member_list}
            # Using POST method to the /edit endpoint
            response_data = make_api_request('post', edit_url, json_data=payload) # Use helper

            print("\nSuccess: Group members updated.")
            if response_data:
                 print(f"API Response: {json.dumps(response_data, indent=2)}") # Display response if available
            # Clear input area on success
            members_input_area.value = ""

        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError updating group members: {e}")
        finally:
            # Re-enable button after attempt
             add_members_button.disabled = False


# --- Define Add Member Tab Layout ---
add_member_tab = widgets.VBox([
    add_load_groups_button,
    add_groups_dropdown,
    widgets.HTML("<br>Enter emails of members to add (one per line):"),
    members_input_area,
    add_members_button,
    widgets.HTML("<hr style='margin: 10px 0;'><b>Status / Results:</b>"),
    add_members_output
])

# --- Connect Handlers ---
add_load_groups_button.on_click(on_add_load_groups_clicked)
add_members_button.on_click(on_add_members_clicked)
# Note: Dropdown and Textarea observers connected via check_add_button_state

# --- Integration Note ---
# The 'add_member_tab' VBox defined above should be assigned as a child
# to the main Tab widget in your dashboard script. For example:
# main_tab_widget.children = [visualize_tab, add_member_tab, ...]

In [4]:
# --- State Variables for this Tab ---
# Using a separate map in case loading is independent from visualize tab
remove_tab_group_name_to_id_map = {}
# Store mapping of display name -> user_id for the currently loaded members
remove_tab_member_display_to_id = {}

# --- Remove Member Tab Widgets ---
remove_load_groups_button = widgets.Button(
    description="Load Groups",
    button_style='primary',
    tooltip='Fetch groups to populate the dropdown',
    icon='refresh'
)

remove_groups_dropdown = widgets.Dropdown(
    description="Target Group:",
    options=[],
    disabled=True,
    style={'description_width': 'initial'}
)

remove_load_current_members_button = widgets.Button(
    description="Load Current Members",
    button_style='info',
    tooltip='Load members for the selected group',
    icon='download',
    disabled=True # Enable only when a group is selected
)

members_to_remove_select = widgets.SelectMultiple(
    description="Select Members to Remove:",
    options=[],
    rows=8, # Adjust number of visible rows
    disabled=True, # Enable after members are loaded
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='auto')
)

remove_members_button = widgets.Button(
    description="Remove Selected Members",
    button_style='danger', # Danger style for removal actions
    tooltip='Remove the selected members from the group',
    icon='user-minus', # FontAwesome icon
    disabled=True # Enable only when members are selected in the list
)

remove_members_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# --- Handler Functions for Remove Member Tab ---

def on_remove_load_groups_clicked(b):
    """Fetches groups and populates the dropdown for the Remove Member tab."""
    global remove_tab_group_name_to_id_map

    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist
    if not current_token:
         with remove_members_output: clear_output(wait=True); print("Authentication required.")
         return
    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with remove_members_output: clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    with remove_members_output:
        clear_output(wait=True)
        print(f"Loading groups from {selected_oasis_name}...")
        # Disable controls
        remove_groups_dropdown.disabled = True
        remove_groups_dropdown.options = []
        remove_groups_dropdown.value = None
        remove_load_current_members_button.disabled = True
        members_to_remove_select.options = []
        members_to_remove_select.disabled = True
        remove_members_button.disabled = True
        remove_tab_group_name_to_id_map = {}

        try:
            groups_url = f"{base_url}/groups"
            groups_response = make_api_request('get', groups_url, params={'page_size': 1000})
            groups = groups_response.get('data', [])
            if not groups: print("No groups found."); return

            remove_tab_group_name_to_id_map = {g['group_name']: g['group_id'] for g in groups}
            dropdown_options = sorted(remove_tab_group_name_to_id_map.keys())
            remove_groups_dropdown.options = dropdown_options
            remove_groups_dropdown.disabled = False
            print(f"Loaded {len(groups)} groups. Select a target group.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"Error loading groups: {e}")
            remove_groups_dropdown.options = []
            remove_groups_dropdown.disabled = True


def on_remove_group_selected(change):
    """Handles group selection: enables Load Members button, resets member list."""
    if change['type'] == 'change' and change['name'] == 'value':
        selected_group_name = change['new']
        # Clear member list and disable subsequent controls
        members_to_remove_select.options = []
        members_to_remove_select.value = [] # Clear selection
        members_to_remove_select.disabled = True
        remove_members_button.disabled = True
        with remove_members_output: clear_output(wait=True)

        if selected_group_name:
            remove_load_current_members_button.disabled = False
            with remove_members_output: print("Group selected. Click 'Load Current Members'.")
        else:
            remove_load_current_members_button.disabled = True


def on_remove_load_current_members_clicked(b):
    """Loads members for the selected group into the SelectMultiple widget."""
    global remove_tab_member_display_to_id

    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist
    if not current_token:
         with remove_members_output: clear_output(wait=True); print("Authentication required.")
         return
    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with remove_members_output: clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    selected_group_name = remove_groups_dropdown.value
    group_id = remove_tab_group_name_to_id_map.get(selected_group_name)
    if not group_id:
         with remove_members_output: clear_output(wait=True); print("Error: No group selected or group ID not found.")
         return

    with remove_members_output:
        clear_output(wait=True)
        print(f"Loading members for group '{selected_group_name}'...")
        # Disable controls while loading
        remove_load_current_members_button.disabled = True
        members_to_remove_select.options = []
        members_to_remove_select.disabled = True
        remove_members_button.disabled = True
        remove_tab_member_display_to_id = {}

        try:
            group_details_url = f"{base_url}/groups/{group_id}"
            group_details = make_api_request('get', group_details_url)
            current_member_ids = group_details.get('members', [])

            if not current_member_ids:
                print("This group currently has no members.")
                remove_load_current_members_button.disabled = False # Re-enable load button
                return

            print(f"Found {len(current_member_ids)} members. Fetching details...")
            member_options = []
            # Fetch details one by one
            for i, user_id in enumerate(current_member_ids):
                print(f"  Fetching user {i+1}/{len(current_member_ids)} (ID: {user_id})...")
                # Assumes fetch_user_details is available from main scope
                member_name = fetch_user_details(user_id, base_url)
                display_text = f"{member_name} (ID: {user_id})"
                member_options.append((display_text, user_id)) # Tuple: (Display Text, Value=user_id)
                remove_tab_member_display_to_id[display_text] = user_id # Store mapping if needed later
                time.sleep(0.05)

            # Sort options alphabetically by display text
            member_options.sort(key=lambda x: x[0])

            members_to_remove_select.options = member_options
            members_to_remove_select.disabled = False # Enable selection list
            print("\nCurrent members loaded. Select members to remove.")

        except (ConnectionError, ValueError, Exception) as e:
             print(f"Error loading members: {e}")
        finally:
             # Always re-enable the load button unless process was successful and list populated
             if not members_to_remove_select.options:
                  remove_load_current_members_button.disabled = False


def on_member_to_remove_selected(change):
    """Enable/disable the remove button based on selection."""
    if change['type'] == 'change' and change['name'] == 'value':
        selected_ids = change['new']
        remove_members_button.disabled = not bool(selected_ids) # Disable if tuple is empty


members_to_remove_select.observe(on_member_to_remove_selected, names='value')


def on_remove_members_clicked(b):
    """Handles removing selected members from the group."""
    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist
    if not current_token:
         with remove_members_output: clear_output(wait=True); print("Authentication required.")
         return
    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with remove_members_output: clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    selected_group_name = remove_groups_dropdown.value
    group_id = remove_tab_group_name_to_id_map.get(selected_group_name)
    if not group_id:
         with remove_members_output: clear_output(wait=True); print("Error: No group selected or group ID not found.")
         return

    # Get the user IDs selected for removal from the widget's value
    selected_ids_for_removal = set(members_to_remove_select.value)
    if not selected_ids_for_removal:
        with remove_members_output: clear_output(wait=True); print("Error: No members selected for removal.")
        return

    with remove_members_output:
        clear_output(wait=True)
        print(f"Removing {len(selected_ids_for_removal)} selected member(s) from group '{selected_group_name}'...")
        remove_members_button.disabled = True # Disable while processing

        try:
            # 1. Fetch ALL Current Members (to ensure we have the latest list)
            print("1. Verifying current member list...")
            group_details_url = f"{base_url}/groups/{group_id}"
            group_details = make_api_request('get', group_details_url)
            all_current_member_ids = group_details.get('members', [])
            print(f"  Found {len(all_current_member_ids)} current members.")

            # 2. Filter List: Keep only those NOT selected for removal
            final_member_list = [user_id for user_id in all_current_member_ids if user_id not in selected_ids_for_removal]
            print(f"  Member list after removal will have {len(final_member_list)} members.")

            # 3. Update Group Members (POST /edit request)
            print("\n2. Updating group membership via POST /edit...")
            edit_url = f"{base_url}/groups/{group_id}/edit"
            payload = {"members": final_member_list}
            response_data = make_api_request('post', edit_url, json_data=payload)

            print("\nSuccess: Group members updated.")
            if response_data:
                 print(f"API Response: {json.dumps(response_data, indent=2)}")

            # Clear the selection list and disable buttons after successful removal
            members_to_remove_select.options = []
            members_to_remove_select.disabled = True
            remove_members_button.disabled = True
            # Keep load members button enabled to allow reloading
            remove_load_current_members_button.disabled = False
            print("\nMember list cleared. Click 'Load Current Members' to refresh.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError removing members: {e}")
            # Re-enable remove button on error if members are still selected
            check_remove_button_state() # Check if selection still valid
        # Note: Button state handled within try/except/finally logic if needed


# Helper to check remove button state (used after errors potentially)
def check_remove_button_state():
     remove_members_button.disabled = not bool(members_to_remove_select.value)

# --- Define Remove Member Tab Layout ---
remove_member_tab = widgets.VBox([
    remove_load_groups_button,
    remove_groups_dropdown,
    remove_load_current_members_button,
    members_to_remove_select,
    remove_members_button,
    widgets.HTML("<hr style='margin: 10px 0;'><b>Status / Results:</b>"),
    remove_members_output
])

# --- Connect Handlers ---
remove_load_groups_button.on_click(on_remove_load_groups_clicked)
remove_groups_dropdown.observe(on_remove_group_selected, names='value')
remove_load_current_members_button.on_click(on_remove_load_current_members_clicked)
remove_members_button.on_click(on_remove_members_clicked)
# Note: SelectMultiple observer connected via on_member_to_remove_selected

# --- Integration Note ---
# The 'remove_member_tab' VBox defined above should be assigned as a child
# to the main Tab widget in your dashboard script. For example:
# main_tab_widget.children = [visualize_tab, add_member_tab, remove_member_tab, ...]

In [5]:
# --- Helper Function for API Calls ---
# Note: If make_api_request is defined globally in your main script, you can remove this.
def make_api_request(method, url, headers=None, params=None, json_data=None, timeout=10):
    """Makes an API request and handles common errors."""
    # Use the global current_token from the main dashboard scope
    if not current_token:
         raise ValueError("Authentication token is missing.")

    # Prepare headers, ensuring the token is included
    request_headers = headers.copy() if headers else {}
    request_headers['Authorization'] = f'Bearer {current_token}'

    try:
        response = requests.request(method, url, headers=request_headers, params=params, json=json_data, timeout=timeout)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        # POST /groups returns 200 OK with body according to user example
        if response.text:
             return response.json()
        return None
    except requests.exceptions.RequestException as e:
        error_message = f"Network/API Error: {e}"
        if e.response is not None:
            try:
                error_detail = e.response.json().get('detail', e.response.text)
                if isinstance(error_detail, list):
                    error_message = f"API Error ({e.response.status_code}): {json.dumps(error_detail)}"
                else:
                    error_message = f"API Error ({e.response.status_code}): {error_detail or e.response.text}"
            except json.JSONDecodeError:
                error_message = f"API Error ({e.response.status_code}): {e.response.text}"
        raise ConnectionError(error_message) from e
    except Exception as e:
        raise Exception(f"Unexpected Error during API request: {e}") from e


# --- Create Group Tab Widgets ---
new_group_name_input = widgets.Text(
    placeholder="Enter name for the new group",
    description="Group Name:",
    style={'description_width': 'initial'}
)

initial_members_input = widgets.Textarea(
    placeholder="Optional: Enter initial member emails, one per line...",
    description="Initial Members:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(height='100px', width='auto')
)

create_group_button = widgets.Button(
    description="Create Group",
    button_style='success',
    tooltip='Create the new group with the specified name and members',
    icon='plus-circle', # FontAwesome icon
    disabled=True # Disabled until group name is entered
)

create_group_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# --- Handler Functions for Create Group Tab ---

def check_create_button_state(*args):
    """Enable Create Group button only if a group name is entered."""
    group_name_entered = bool(new_group_name_input.value.strip())
    create_group_button.disabled = not group_name_entered

# Observe changes in group name input to control button state
new_group_name_input.observe(check_create_button_state, names='value')


def on_create_group_clicked(b):
    """Handles creating a new group."""
    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist in the parent scope
    if not current_token:
         with create_group_output:
            clear_output(wait=True); print("Authentication required.")
         return

    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with create_group_output:
            clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    # Get group name and initial emails
    group_name = new_group_name_input.value.strip()
    emails_text = initial_members_input.value.strip()

    if not group_name:
        with create_group_output: clear_output(wait=True); print("Error: Group Name cannot be empty.")
        return

    initial_emails = [email.strip() for email in emails_text.splitlines() if email.strip()]

    with create_group_output:
        clear_output(wait=True)
        print(f"Creating group '{group_name}'...")
        create_group_button.disabled = True # Disable while processing

        found_user_ids = []
        not_found_emails = []

        # 1. Look up User IDs if initial emails were provided
        if initial_emails:
            print("\n1. Looking up initial members by email...")
            users_url = f"{base_url}/users"
            for i, email in enumerate(initial_emails):
                print(f"  Checking email {i+1}/{len(initial_emails)}: {email}...")
                try:
                    user_response = make_api_request('get', users_url, params={'email': email}) # Use helper
                    user_data = user_response.get('data', []) if user_response else []

                    if user_data and len(user_data) > 0:
                        user_id = user_data[0].get('user_id')
                        if user_id:
                            found_user_ids.append(user_id)
                            print(f"    Found User ID: {user_id}")
                        else:
                            not_found_emails.append(email)
                            print("    Error: Found user data but no user_id.")
                    else:
                        not_found_emails.append(email)
                        print("    User not found.")
                except (ConnectionError, ValueError, Exception) as e:
                    print(f"    Error looking up email {email}: {e}")
                    not_found_emails.append(f"{email} (Lookup Error)")
                time.sleep(0.05) # Small delay

            # Report users not found
            if not_found_emails:
                print("\nWarning: Could not find or process the following initial members:")
                for email in not_found_emails:
                    print(f"- {email}")
                print("Group will be created without these users.")

            if not found_user_ids and initial_emails:
                 print("\nWarning: None of the initial member emails were found. Creating group without members.")

        # 2. Prepare Payload for POST /groups
        payload = {'group_name': group_name}
        if found_user_ids:
            # Ensure uniqueness just in case emails mapped to same ID
            payload['members'] = list(set(found_user_ids))

        # 3. Create Group (POST request)
        print(f"\n2. Creating group '{group_name}' via POST...")
        try:
            groups_url = f"{base_url}/groups"
            response_data = make_api_request('post', groups_url, json_data=payload) # Use helper

            print("\nSuccess: Group created.")
            if response_data:
                 print(f"API Response: {json.dumps(response_data, indent=2)}")
                 # Optionally update group lists in other tabs if needed, e.g., by calling their load functions
            # Clear input fields on success
            new_group_name_input.value = ""
            initial_members_input.value = ""

        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError creating group: {e}")
            # Check for specific errors like 'group already exists' if API provides hints
        finally:
            # Re-enable button after attempt, but only if group name field is still populated
            # (it was cleared on success)
            check_create_button_state()


# --- Define Create Group Tab Layout ---
create_group_tab = widgets.VBox([
    new_group_name_input,
    initial_members_input,
    create_group_button,
    widgets.HTML("<hr style='margin: 10px 0;'><b>Status / Results:</b>"),
    create_group_output
])

# --- Connect Handlers ---
create_group_button.on_click(on_create_group_clicked)
# Note: Group name input observer connected via check_create_button_state

# --- Integration Note ---
# The 'create_group_tab' VBox defined above should be assigned as a child
# to the main Tab widget in your dashboard script. For example:
# main_tab_widget.children = [visualize_tab, add_member_tab, remove_member_tab, create_group_tab, ...]


In [6]:
# --- State Variables for this Tab ---
# Using a separate map in case loading is independent from other tabs
delete_tab_group_name_to_id_map = {}

# --- Delete Group Tab Widgets ---
delete_load_groups_button = widgets.Button(
    description="Load Groups",
    button_style='primary',
    tooltip='Fetch groups to populate the dropdown',
    icon='refresh'
)

delete_groups_dropdown = widgets.Dropdown(
    description="Group to Delete:",
    options=[],
    disabled=True,
    style={'description_width': 'initial'}
)

# ADDED: Text input for confirmation
delete_confirmation_input = widgets.Text(
    placeholder="Type the group name exactly to confirm",
    description="Confirm Name:",
    style={'description_width': 'initial'},
    disabled=True # Enable only when a group is selected
)

# Final deletion button - enabled after group selection AND name confirmation
delete_confirm_button = widgets.Button(
    description="Delete This Group", # Simpler text now
    button_style='danger',
    tooltip='Permanently delete the group after confirming name',
    icon='trash-2', # Lucide/FontAwesome icon for trash
    disabled=True # Enable only when group selected and name matches
)

delete_group_output = widgets.Output(
    layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='10px 0 0 0', min_height='100px')
)

# --- Handler Functions for Delete Group Tab ---

def on_delete_load_groups_clicked(b):
    """Fetches groups and populates the dropdown for the Delete Group tab."""
    global delete_tab_group_name_to_id_map

    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist
    if not current_token:
         with delete_group_output: clear_output(wait=True); print("Authentication required.")
         return
    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with delete_group_output: clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    with delete_group_output:
        clear_output(wait=True)
        print(f"Loading groups from {selected_oasis_name}...")
        # Disable controls
        delete_groups_dropdown.disabled = True
        delete_groups_dropdown.options = []
        delete_groups_dropdown.value = None
        delete_confirmation_input.value = ""
        delete_confirmation_input.disabled = True
        delete_confirm_button.disabled = True
        delete_tab_group_name_to_id_map = {}

        try:
            groups_url = f"{base_url}/groups"
            groups_response = make_api_request('get', groups_url, params={'page_size': 1000})
            groups = groups_response.get('data', [])
            if not groups: print("No groups found."); return

            delete_tab_group_name_to_id_map = {g['group_name']: g['group_id'] for g in groups}
            dropdown_options = sorted(delete_tab_group_name_to_id_map.keys())
            delete_groups_dropdown.options = dropdown_options
            delete_groups_dropdown.disabled = False
            print(f"Loaded {len(groups)} groups. Select group to delete.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"Error loading groups: {e}")
            delete_groups_dropdown.options = []
            delete_groups_dropdown.disabled = True


def check_delete_confirmation(*args):
    """Enable delete button only if selected group name matches typed text."""
    selected_group = delete_groups_dropdown.value
    typed_name = delete_confirmation_input.value

    if selected_group and typed_name == selected_group:
        delete_confirm_button.disabled = False
    else:
        delete_confirm_button.disabled = True


def on_delete_group_selected(change):
    """Handles group selection: enables confirmation input and prompts user."""
    if change['type'] == 'change' and change['name'] == 'value':
        selected_group_name = change['new']

        # Always clear confirmation input and disable final button on change
        delete_confirmation_input.value = ""
        delete_confirm_button.disabled = True
        with delete_group_output: clear_output(wait=True) # Clear previous messages

        if selected_group_name:
            delete_confirmation_input.disabled = False # Enable text input
            with delete_group_output:
                print(f"Selected '{selected_group_name}' for deletion.")
                print("WARNING: Deletion is permanent.")
                print(f"To confirm, please type the group name ('{selected_group_name}') into the 'Confirm Name' box below.")
        else:
            delete_confirmation_input.disabled = True # Disable text input


def on_delete_confirm_button_clicked(b):
    """Handles the final deletion of the selected group after confirmation."""
    # Assumes 'current_token', 'oasis_dropdown', 'oasis_options' exist
    if not current_token:
         with delete_group_output: clear_output(wait=True); print("Authentication required.")
         return
    selected_oasis_name = oasis_dropdown.value
    base_url = oasis_options.get(selected_oasis_name)
    if not base_url:
         with delete_group_output: clear_output(wait=True); print("Error: Invalid Oasis selected.")
         return

    selected_group_name = delete_groups_dropdown.value
    # Double check confirmation text matches selected group before proceeding
    if not selected_group_name or delete_confirmation_input.value != selected_group_name:
         with delete_group_output:
             clear_output(wait=True)
             print("Error: Confirmation failed. Selected group name does not match typed text.")
             delete_confirm_button.disabled = True # Ensure button is disabled
         return

    group_id = delete_tab_group_name_to_id_map.get(selected_group_name)
    if not group_id:
         # This case should be unlikely if dropdown is populated correctly
         with delete_group_output:
             clear_output(wait=True)
             print(f"Error: Could not find ID for selected group '{selected_group_name}'.")
             delete_confirm_button.disabled = True
         return

    with delete_group_output:
        clear_output(wait=True)
        print(f"Attempting to delete group '{selected_group_name}' (ID: {group_id})...")
        delete_confirm_button.disabled = True # Disable while processing
        delete_confirmation_input.disabled = True # Disable input too

        try:
            # *** API Call: DELETE /groups/{group_id} ***
            delete_url = f"{base_url}/groups/{group_id}"
            make_api_request('delete', delete_url) # Use helper. Expects 204 No Content on success.

            print(f"\nSuccess: Group '{selected_group_name}' deleted.")

            # Reset state after successful deletion
            delete_groups_dropdown.value = None # Clear selection
            delete_confirmation_input.value = "" # Clear confirmation input
            # Remove deleted group from local map and dropdown options
            if selected_group_name in delete_tab_group_name_to_id_map:
                del delete_tab_group_name_to_id_map[selected_group_name]
            current_options = list(delete_groups_dropdown.options)
            if selected_group_name in current_options:
                current_options.remove(selected_group_name)
                delete_groups_dropdown.options = current_options

            # Keep input disabled, button remains disabled as selection is None
            print("\nPlease reload group lists in other tabs if necessary.")

        except (ConnectionError, ValueError, Exception) as e:
            print(f"\nError deleting group: {e}")
            # Re-enable input and check button state on failure
            delete_confirmation_input.disabled = False
            check_delete_confirmation() # Check if button should be re-enabled
        # No finally needed as button state handled


# --- Define Delete Group Tab Layout ---
delete_group_tab = widgets.VBox([
    delete_load_groups_button,
    delete_groups_dropdown,
    delete_confirmation_input, # Added confirmation input
    delete_confirm_button,
    widgets.HTML("<hr style='margin: 10px 0;'><b>Status / Results:</b>"),
    delete_group_output
])

# --- Connect Handlers ---
delete_load_groups_button.on_click(on_delete_load_groups_clicked)
delete_groups_dropdown.observe(on_delete_group_selected, names='value')
delete_confirmation_input.observe(check_delete_confirmation, names='value') # Observe text input
delete_confirm_button.on_click(on_delete_confirm_button_clicked)

# --- Integration Note ---
# The 'delete_group_tab' VBox defined above should be assigned as a child
# to the main Tab widget in your dashboard script. For example:
# main_tab_widget.children = [..., create_group_tab, delete_group_tab]


In [ ]:
# --- Tab Widgets (Placeholders remain the same) ---
#visualize_tab = widgets.VBox([widgets.Label("Controls for visualizing group members will go here.")])
#add_member_tab = widgets.VBox([widgets.Label("Controls for adding members to a group will go here.")])
#remove_member_tab = widgets.VBox([widgets.Label("Controls for removing members from a group will go here.")])
#create_group_tab = widgets.VBox([widgets.Label("Controls for creating a new group will go here.")])
#delete_group_tab = widgets.VBox([widgets.Label("Controls for deleting an existing group will go here.")])

tab_widget = widgets.Tab()
tab_widget.children = [visualize_tab, add_member_tab, remove_member_tab, create_group_tab, delete_group_tab]
tab_widget.set_title(0, 'Visualize Members')
tab_widget.set_title(1, 'Add Members')
tab_widget.set_title(2, 'Remove Members')
tab_widget.set_title(3, 'Create Group')
tab_widget.set_title(4, 'Delete Group')

app_layout = widgets.Layout(
    max_width="1000px",  # Set your desired max width
    margin="0 auto",  # Center the layout
    # You might add padding, borders etc. here too
    padding='15px'
)

# --- Main Dashboard Layout ---
dashboard_layout = widgets.VBox([
    widgets.HTML("<h1>NOMAD Group Management</h1>"),
    general_settings_box,
    tab_widget
],
    layout=app_layout
)

In [8]:
# --- Display the Dashboard ---
# Ensure this line is executed in your notebook cell
display(dashboard_layout)